# Synchronous Hybrid Neuroevolution Notebook

This notebook implements the complete hybrid neuroevolution process synchronously, without the need for databases or external Kafka services. The system combines genetic algorithms with convolutional neural networks to evolve optimal architectures.

## Main Features:
- **Hybrid genetic algorithm**: Combines architecture and weight evolution
- **Synchronous processing**: Complete workflow executed in a single session
- **Configurable dataset**: Supports MNIST by default or custom dataset
- **Intelligent stopping criteria**: By target fitness or maximum generations
- **Complete visualization**: Shows progress and final best architecture

## Objectives:
1. Create initial population of CNN architectures
2. Evaluate fitness of each individual
3. Select best architectures (top 50%)
4. Apply crossover and mutation to create new generation
5. Repeat process until convergence
6. Display the best architecture found

## 1. Required Libraries Import

In [ ]:
# Install all necessary libraries
import subprocess
import sys

def install_package(package):
    """Installs a package using pip if not available."""
    try:
        __import__(package.split('==')[0].split('[')[0])
        print(f"OK {package.split('==')[0]} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"OK {package} installed correctly")

# List of required packages
required_packages = [
    "torch>=2.0.0",
    "torchvision>=0.15.0",
    "numpy>=1.21.0",
    "matplotlib>=3.5.0",
    "seaborn>=0.11.0",
    "tqdm>=4.64.0",
    "jupyter>=1.0.0",
    "ipywidgets>=8.0.0"
]

print("Starting dependency installation for Hybrid Neuroevolution...")
print("=" * 60)

for package in required_packages:
    install_package(package)

print("\nAll dependencies have been verified/installed")
print("Restart the kernel if this is the first time installing torch")
print("=" * 60)

# Verify PyTorch installation
try:
    import torch
    print(f"\nPyTorch {torch.__version__} installed correctly")
    print(f"CUDA available: {'Yes' if torch.cuda.is_available() else 'No'}")
    if torch.cuda.is_available():
        print(f"GPU detected: {torch.cuda.get_device_name(0)}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")
except ImportError:
    print("ERROR: PyTorch could not be installed correctly")
    print("Try installing manually with: pip install torch torchvision")

In [ ]:
# Main imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Scientific libraries
import numpy as np
import random
import copy
import json
import os
import time  # Added for timeout functionality
from typing import Dict, List, Tuple, Any
from datetime import datetime
import uuid
import threading  # Added for concurrent model training

# Visualization and progress
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## 2. System Configuration and Parameters

In [ ]:
# Main genetic algorithm configuration (updated for adaptive mutation & moderate elitism)
CONFIG = {
    # Genetic algorithm parameters
    'population_size': 10,            # Population size
    'max_generations': 30,            # Maximum number of generations
    'fitness_threshold': 99.9,        # Target fitness (% accuracy)

    # Adaptive mutation parameters
    'base_mutation_rate': 0.35,       # Starting mutation rate (moderate)
    'mutation_rate_min': 0.10,        # Lower bound for adaptive mutation
    'mutation_rate_max': 0.80,        # Upper bound for adaptive mutation
    'current_mutation_rate': 0.35,    # Will be updated dynamically each generation

    'crossover_rate': 0.99,           # Crossover rate
    'elite_percentage': 0.2,          # Moderate elitism (20%) instead of 40%

    # Dataset selection
    'dataset': 'MNIST',             # Options: 'MNIST', 'CIFAR10', 'CUSTOM'

    # Dataset parameters (auto-configured based on dataset)
    'num_channels': 3,                # Input channels (1=grayscale, 3=RGB)
    'px_h': 32,                       # Image height
    'px_w': 32,                       # Image width
    'num_classes': 10,                # Number of classes
    'batch_size': 128,                # Batch size
    'test_split': 0.35,               # Validation percentage (for CUSTOM)

    # Training parameters
    'num_epochs': 12,                 # Max training epochs per evaluation (may stop earlier)
    'learning_rate': 0.01,            # Base learning rate (used only if genome doesn't override)
    'early_stopping_patience': 1000,  # Max batches per epoch (quick partial epoch)

    # Epoch-level early stopping (new)
    'epoch_patience': 3,              # Stop if no significant improvement after N evaluations
    'improvement_threshold': 0.2,     # Minimum (absolute) accuracy gain (%) to reset patience

    # Allowed architecture range
    'min_conv_layers': 1,
    'max_conv_layers': 7,
    'min_fc_layers': 1,
    'max_fc_layers': 7,
    'min_filters': 2,
    'max_filters': 256,
    'min_fc_nodes': 128,
    'max_fc_nodes': 2048,

    # Custom dataset configuration (only used if dataset='CUSTOM')
    'dataset_path': None,             # Custom dataset path
}

# Dataset configurations
DATASET_CONFIGS = {
    'MNIST': {
        'num_channels': 1,
        'px_h': 28,
        'px_w': 28,
        'num_classes': 10,
        'normalization': {'mean': (0.1307,), 'std': (0.3081,)}
    },
    'CIFAR10': {
        'num_channels': 3,
        'px_h': 32,
        'px_w': 32,
        'num_classes': 10,
        'normalization': {'mean': (0.4914, 0.4822, 0.4465), 'std': (0.2023, 0.1994, 0.2010)}
    },
    'CUSTOM': {
        'num_channels': 1,  # Default, will be overridden
        'px_h': 28,         # Default, will be overridden
        'px_w': 28,         # Default, will be overridden
        'num_classes': 10,  # Default, will be overridden
        'normalization': {'mean': (0.5,), 'std': (0.5,)}
    }
}

# Auto-configure based on selected dataset
def configure_dataset(config, dataset_name):
    """Auto-configures dataset parameters based on selected dataset."""
    if dataset_name in DATASET_CONFIGS:
        dataset_config = DATASET_CONFIGS[dataset_name]
        config['num_channels'] = dataset_config['num_channels']
        config['px_h'] = dataset_config['px_h']
        config['px_w'] = dataset_config['px_w']
        config['num_classes'] = dataset_config['num_classes']
        config['_normalization'] = dataset_config['normalization']
    return config

# Configure the selected dataset
CONFIG = configure_dataset(CONFIG, CONFIG['dataset'])

# Activation function mapping
ACTIVATION_FUNCTIONS = {
    'relu': nn.ReLU,
    'leaky_relu': nn.LeakyReLU,
    'tanh': nn.Tanh,
    'sigmoid': nn.Sigmoid,
    'selu': nn.SELU,
}

# Optimizer mapping
OPTIMIZERS = {
    'adam': optim.Adam,
    'adamw': optim.AdamW,
    'sgd': optim.SGD,
    'rmsprop': optim.RMSprop,
}

print("Configuration loaded (adaptive mutation enabled):")
print(f"   Selected dataset: {CONFIG['dataset']}")
for key, value in CONFIG.items():
    if not key.startswith('_'):  # Hide internal config
        print(f"   {key}: {value}")
print(f"\nAvailable activation functions: {list(ACTIVATION_FUNCTIONS.keys())}")
print(f"Available optimizers: {list(OPTIMIZERS.keys())}")
print(f"Available datasets: {list(DATASET_CONFIGS.keys())}")

### Información sobre Datasets Disponibles

**MNIST**: 
- Dígitos escritos a mano (0-9)
- Imágenes en escala de grises (1 canal)
- Tamaño: 28x28 píxeles
- Dificultad: **Fácil** - Ideal para pruebas rápidas
- Fitness objetivo recomendado: >95%

**CIFAR-10**: 
- Objetos del mundo real (aviones, coches, pájaros, etc.)
- Imágenes en color (3 canales RGB)
- Tamaño: 32x32 píxeles
- Dificultad: **Media-Alta** - Más realista y desafiante
- Fitness objetivo recomendado: >80%
- Clases: plane, car, bird, cat, deer, dog, frog, horse, ship, truck

**CUSTOM**: 
- Tu propio dataset
- Configuración manual requerida
- Estructura de carpetas por clase

## 3. Dataset Loading and Preprocessing

In [ ]:
def load_dataset(config: dict) -> Tuple[DataLoader, DataLoader]:
    """
    Loads the dataset according to configuration.
    Returns train_loader and test_loader.
    """
    
    dataset_type = config['dataset']
    
    if dataset_type == 'CUSTOM' and config['dataset_path']:
        print(f"Loading custom dataset from: {config['dataset_path']}")
        
        # Transformations for custom dataset
        if config['num_channels'] == 1:
            normalize = transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        else:
            normalize = transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        
        transform = transforms.Compose([
            transforms.Resize((config['px_h'], config['px_w'])),
            transforms.ToTensor(),
            normalize
        ])
        
        # Load dataset from folders organized by class
        full_dataset = datasets.ImageFolder(root=config['dataset_path'], transform=transform)
        
        # Split into train and test
        train_size = int((1 - config['test_split']) * len(full_dataset))
        test_size = len(full_dataset) - train_size
        train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
        
        print(f"Custom dataset loaded:")
        print(f"   Classes found: {len(full_dataset.classes)}")
        print(f"   Total samples: {len(full_dataset)}")
        
    elif dataset_type == 'MNIST':
        print("Loading MNIST dataset...")
        
        # Transformations for MNIST
        transform = transforms.Compose([
            transforms.Resize((config['px_h'], config['px_w'])),
            transforms.ToTensor(),
            transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        ])
        
        # Load MNIST
        train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
        
        print(f"MNIST dataset loaded:")
        print(f"   Classes: {len(train_dataset.classes)}")
        print(f"   Training samples: {len(train_dataset)}")
        print(f"   Test samples: {len(test_dataset)}")
        
    elif dataset_type == 'CIFAR10':
        print("Loading CIFAR-10 dataset...")
        
        # Transformations for CIFAR-10
        transform_train = transforms.Compose([
            transforms.RandomHorizontalFlip(),  # Data augmentation for training
            transforms.RandomCrop(32, padding=4),
            transforms.ToTensor(),
            transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        ])
        
        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        ])
        
        # Load CIFAR-10
        train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
        test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
        
        print(f"CIFAR-10 dataset loaded:")
        print(f"   Classes: {train_dataset.classes}")
        print(f"   Training samples: {len(train_dataset)}")
        print(f"   Test samples: {len(test_dataset)}")
        
    else:
        raise ValueError(f"Dataset '{dataset_type}' not supported. Available: MNIST, CIFAR10, CUSTOM")
    
    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config['batch_size'], 
        shuffle=True,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=config['batch_size'], 
        shuffle=False,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    return train_loader, test_loader

# Load the dataset
train_loader, test_loader = load_dataset(CONFIG)

# Get a sample to verify dimensions
sample_batch = next(iter(train_loader))
sample_data, sample_labels = sample_batch
print(f"\nDataset loaded successfully:")
print(f"   Batch shape: {sample_data.shape}")
print(f"   Data type: {sample_data.dtype}")
print(f"   Device: {sample_data.device}")
print(f"   Value range: [{sample_data.min():.3f}, {sample_data.max():.3f}]")

# Show some class information for CIFAR-10
if CONFIG['dataset'] == 'CIFAR10':
    cifar10_classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    print(f"   CIFAR-10 classes: {cifar10_classes}")
    unique_labels = torch.unique(sample_labels)
    print(f"   Labels in this batch: {unique_labels.tolist()}")

## 4. Neural Network Architecture Definition

In [ ]:
class EvolvableCNN(nn.Module):
    """
    Evolvable CNN class that can be dynamically configured
    according to genome parameters.
    """
    
    def __init__(self, genome: dict, config: dict):
        super(EvolvableCNN, self).__init__()
        self.genome = genome
        self.config = config
        
        # Build convolutional layers
        self.conv_layers = self._build_conv_layers()
        
        # Calculate output size after convolutions
        self.conv_output_size = self._calculate_conv_output_size()
        
        # Build fully connected layers
        self.fc_layers = self._build_fc_layers()
        
    def _build_conv_layers(self) -> nn.ModuleList:
        """Builds convolutional layers according to genome."""
        layers = nn.ModuleList()
        
        in_channels = self.config['num_channels']
        
        for i in range(self.genome['num_conv_layers']):
            out_channels = self.genome['filters'][i]
            kernel_size = self.genome['kernel_sizes'][i]
            
            # Convolutional layer
            conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
            layers.append(conv)
            
            # Batch normalization
            layers.append(nn.BatchNorm2d(out_channels))
            
            # Activation function
            activation_name = self.genome['activations'][i % len(self.genome['activations'])]
            activation_func = ACTIVATION_FUNCTIONS[activation_name]()
            layers.append(activation_func)
            
            # Max pooling (except in last layer)
            if i < self.genome['num_conv_layers'] - 1:
                layers.append(nn.MaxPool2d(2, 2))
            else:
                layers.append(nn.MaxPool2d(2, 1))  # Stride 1 in last layer
            
            in_channels = out_channels
            
        return layers
    
    def _calculate_conv_output_size(self) -> int:
        """Calculates output size after convolutional layers."""
        # Create dummy tensor to calculate size
        dummy_input = torch.zeros(1, self.config['num_channels'], 
                                 self.config['px_h'], self.config['px_w'])
        
        # Pass through convolutional layers
        x = dummy_input
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten and get size
        return x.view(-1).shape[0]
    
    def _build_fc_layers(self) -> nn.ModuleList:
        """Builds fully connected layers."""
        layers = nn.ModuleList()
        
        input_size = self.conv_output_size
        
        for i in range(self.genome['num_fc_layers']):
            output_size = self.genome['fc_nodes'][i]
            
            # Linear layer
            layers.append(nn.Linear(input_size, output_size))
            
            # Dropout if not last layer
            if i < self.genome['num_fc_layers'] - 1:
                layers.append(nn.Dropout(self.genome['dropout_rate']))
            
            input_size = output_size
        
        # Final classification layer
        layers.append(nn.Linear(input_size, self.config['num_classes']))
        
        return layers
    
    def forward(self, x):
        """Forward pass of the network."""
        # Convolutional layers
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        for i, layer in enumerate(self.fc_layers):
            x = layer(x)
            # Apply activation except on last layer
            if i < len(self.fc_layers) - 1 and not isinstance(layer, nn.Dropout):
                x = F.relu(x)
        
        return x
    
    def get_architecture_summary(self) -> str:
        """Returns an architecture summary."""
        summary = []
        summary.append(f"Conv Layers: {self.genome['num_conv_layers']}")
        summary.append(f"Filters: {self.genome['filters']}")
        summary.append(f"Kernel Sizes: {self.genome['kernel_sizes']}")
        summary.append(f"FC Layers: {self.genome['num_fc_layers']}")
        summary.append(f"FC Nodes: {self.genome['fc_nodes']}")
        summary.append(f"Activations: {self.genome['activations']}")
        summary.append(f"Dropout: {self.genome['dropout_rate']:.3f}")
        summary.append(f"Optimizer: {self.genome['optimizer']}")
        summary.append(f"Learning Rate: {self.genome['learning_rate']:.4f}")
        return " | ".join(summary)

print("EvolvableCNN class defined correctly")

## 5. Genetic Algorithm Components

In [ ]:
def create_random_genome(config: dict) -> dict:
    """Creates a random genome within specified ranges."""
    # Number of layers
    num_conv_layers = random.randint(config['min_conv_layers'], config['max_conv_layers'])
    num_fc_layers = random.randint(config['min_fc_layers'], config['max_fc_layers'])

    # Filters for each convolutional layer
    filters = [random.randint(config['min_filters'], config['max_filters']) for _ in range(num_conv_layers)]

    # Kernel sizes
    kernel_sizes = [random.choice([3, 5, 7]) for _ in range(num_conv_layers)]

    # Nodes in fully connected layers
    fc_nodes = [random.randint(config['min_fc_nodes'], config['max_fc_nodes']) for _ in range(num_fc_layers)]

    # Activation functions for each layer
    activations = [random.choice(list(ACTIVATION_FUNCTIONS.keys())) for _ in range(max(num_conv_layers, num_fc_layers))]

    # Other parameters
    dropout_rate = random.uniform(0.1, 0.5)
    learning_rate = random.choice([0.001, 0.0001, 0.01, 0.005])
    optimizer = random.choice(list(OPTIMIZERS.keys()))

    genome = {
        'num_conv_layers': num_conv_layers,
        'num_fc_layers': num_fc_layers,
        'filters': filters,
        'kernel_sizes': kernel_sizes,
        'fc_nodes': fc_nodes,
        'activations': activations,
        'dropout_rate': dropout_rate,
        'learning_rate': learning_rate,
        'optimizer': optimizer,
        'fitness': 0.0,
        'id': str(uuid.uuid4())[:8]
    }
    return genome

def mutate_genome(genome: dict, config: dict) -> dict:
    """Applies mutation to a genome using adaptive mutation rate."""
    mutated_genome = copy.deepcopy(genome)
    mutation_rate = config['current_mutation_rate']  # adaptive

    # Mutate number of convolutional layers
    if random.random() < mutation_rate:
        mutated_genome['num_conv_layers'] = random.randint(config['min_conv_layers'], config['max_conv_layers'])
        num_conv = mutated_genome['num_conv_layers']
        mutated_genome['filters'] = mutated_genome['filters'][:num_conv]
        mutated_genome['kernel_sizes'] = mutated_genome['kernel_sizes'][:num_conv]
        while len(mutated_genome['filters']) < num_conv:
            mutated_genome['filters'].append(random.randint(config['min_filters'], config['max_filters']))
        while len(mutated_genome['kernel_sizes']) < num_conv:
            mutated_genome['kernel_sizes'].append(random.choice([1, 3, 5, 7]))

    # Mutate filters
    for i in range(len(mutated_genome['filters'])):
        if random.random() < mutation_rate:
            mutated_genome['filters'][i] = random.randint(config['min_filters'], config['max_filters'])

    # Mutate kernel sizes
    for i in range(len(mutated_genome['kernel_sizes'])):
        if random.random() < mutation_rate:
            mutated_genome['kernel_sizes'][i] = random.choice([1, 3, 5, 7])

    # Mutate number of FC layers
    if random.random() < mutation_rate:
        mutated_genome['num_fc_layers'] = random.randint(config['min_fc_layers'], config['max_fc_layers'])
        num_fc = mutated_genome['num_fc_layers']
        mutated_genome['fc_nodes'] = mutated_genome['fc_nodes'][:num_fc]
        while len(mutated_genome['fc_nodes']) < num_fc:
            mutated_genome['fc_nodes'].append(random.randint(config['min_fc_nodes'], config['max_fc_nodes']))

    # Mutate FC nodes
    for i in range(len(mutated_genome['fc_nodes'])):
        if random.random() < mutation_rate:
            mutated_genome['fc_nodes'][i] = random.randint(config['min_fc_nodes'], config['max_fc_nodes'])

    # Mutate activation functions
    for i in range(len(mutated_genome['activations'])):
        if random.random() < mutation_rate:
            mutated_genome['activations'][i] = random.choice(list(ACTIVATION_FUNCTIONS.keys()))

    # Mutate dropout
    if random.random() < mutation_rate:
        mutated_genome['dropout_rate'] = random.uniform(0.1, 0.8)

    # Mutate learning rate
    if random.random() < mutation_rate:
        mutated_genome['learning_rate'] = random.choice([0.001, 0.0001, 0.01, 0.005, 0.000001, 0.05, 0.00005, 0.0005])

    # Mutate optimizer
    if random.random() < mutation_rate:
        mutated_genome['optimizer'] = random.choice(list(OPTIMIZERS.keys()))

    mutated_genome['id'] = str(uuid.uuid4())[:8]
    mutated_genome['fitness'] = 0.0
    return mutated_genome

def crossover_genomes(parent1: dict, parent2: dict, config: dict) -> Tuple[dict, dict]:
    """Performs crossover between two genomes."""
    if random.random() > config['crossover_rate']:
        return copy.deepcopy(parent1), copy.deepcopy(parent2)

    child1 = copy.deepcopy(parent1)
    child2 = copy.deepcopy(parent2)

    # Crossover scalar parameters
    for key in ['num_conv_layers', 'num_fc_layers', 'dropout_rate', 'learning_rate', 'optimizer']:
        if random.random() < 0.5:
            child1[key], child2[key] = child2[key], child1[key]

    # Crossover lists (random cut point)
    for list_key in ['filters', 'kernel_sizes', 'fc_nodes', 'activations']:
        if random.random() < 0.5:
            list1 = child1[list_key]
            list2 = child2[list_key]
            if len(list1) > 1 and len(list2) > 1:
                point1 = random.randint(1, len(list1) - 1)
                point2 = random.randint(1, len(list2) - 1)
                child1[list_key] = list1[:point1] + list2[point2:]
                child2[list_key] = list2[:point2] + list1[point1:]

    child1['id'] = str(uuid.uuid4())[:8]
    child2['id'] = str(uuid.uuid4())[:8]
    child1['fitness'] = 0.0
    child2['fitness'] = 0.0
    return child1, child2

print("Genetic functions updated for adaptive mutation")

## 6. Hybrid Neuroevolution Implementation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class HybridNeuroevolution:
    """Main class that implements hybrid neuroevolution with adaptive mutation & epoch interleaved eval + Threading."""

    def __init__(self, config: dict, train_loader: DataLoader, test_loader: DataLoader):
        self.config = config
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.population = []
        self.generation = 0
        self.best_individual = None
        self.fitness_history = []
        self.generation_stats = []
        # Threading configuration
        self.max_concurrent_training = config.get('max_concurrent_training', 2)  # Configurable, default 2
        self.training_semaphore = threading.Semaphore(self.max_concurrent_training)
        self.fitness_lock = threading.Lock()  # Para acceso seguro a variables compartidas

    def initialize_population(self):
        print(f"Initializing population of {self.config['population_size']} individuals...")
        self.population = [create_random_genome(self.config) for _ in range(self.config['population_size'])]
        print(f"Population initialized with {len(self.population)} individuals")

    def _train_one_epoch(self, model, optimizer, criterion, genome_id: str, epoch: int, thread_name: str):
        """Train one epoch with error handling."""
        try:
            model.train()
            running_loss = 0.0
            batch_count = 0
            max_batches = min(len(self.train_loader), self.config['early_stopping_patience'])
            
            for data, target in self.train_loader:
                try:
                    data, target = data.to(device), target.to(device)
                    optimizer.zero_grad()
                    output = model(data)
                    loss = criterion(output, target)
                    
                    # Check for invalid loss
                    if torch.isnan(loss) or torch.isinf(loss):
                        print(f"{thread_name}: WARNING - Invalid loss detected, skipping batch")
                        continue
                    
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                    batch_count += 1
                    
                    if batch_count >= max_batches:
                        break
                        
                except Exception as batch_error:
                    print(f"{thread_name}: ERROR in training batch: {batch_error}")
                    continue
            
            if batch_count == 0:
                print(f"{thread_name}: ERROR - No valid batches processed in training")
                return None
            
            avg_loss = running_loss / batch_count
            print(f"{thread_name}: Train Epoch {epoch}: loss={avg_loss:.4f} ({batch_count} batches)")
            return avg_loss
            
        except Exception as e:
            print(f"{thread_name}: ERROR in _train_one_epoch: {e}")
            return None

    def _evaluate(self, model, criterion, genome_id: str, epoch: int, thread_name: str):
        """Evaluate model with error handling."""
        try:
            model.eval()
            correct = 0
            total = 0
            eval_batches = 0
            max_eval_batches = min(len(self.test_loader), 20)
            total_eval_loss = 0.0
            
            with torch.no_grad():
                for data, target in self.test_loader:
                    try:
                        data, target = data.to(device), target.to(device)
                        output = model(data)
                        loss = criterion(output, target)
                        
                        # Check for invalid loss
                        if torch.isnan(loss) or torch.isinf(loss):
                            print(f"{thread_name}: WARNING - Invalid eval loss detected, skipping batch")
                            continue
                        
                        total_eval_loss += loss.item()
                        _, predicted = torch.max(output, 1)
                        total += target.size(0)
                        correct += (predicted == target).sum().item()
                        eval_batches += 1
                        
                        if eval_batches >= max_eval_batches:
                            break
                            
                    except Exception as batch_error:
                        print(f"{thread_name}: ERROR in evaluation batch: {batch_error}")
                        continue
            
            if eval_batches == 0 or total == 0:
                print(f"{thread_name}: ERROR - No valid evaluation batches processed")
                return None, None
            
            accuracy = 100.0 * correct / total
            avg_eval_loss = total_eval_loss / eval_batches
            print(f"{thread_name}: Eval  Epoch {epoch}: acc={accuracy:.2f}% loss={avg_eval_loss:.4f} ({eval_batches} batches)")
            return accuracy, avg_eval_loss
            
        except Exception as e:
            print(f"{thread_name}: ERROR in _evaluate: {e}")
            return None, None

    def evaluate_fitness(self, genome: dict, thread_name: str = "MainThread") -> float:
        """Thread-safe fitness evaluation method with robust error handling."""
        evaluation_start = time.time()
        max_evaluation_time = 180  # 3 minutes max per fitness evaluation
        
        try:
            # Validate genome before creating model
            if not self._validate_genome(genome):
                print(f"{thread_name}: ERROR - Invalid genome structure for {genome['id']}")
                return 0.0
            
            print(f"{thread_name}: Creating model for genome {genome['id']}...")
            model = EvolvableCNN(genome, self.config).to(device)
            
            # Validate model creation
            if model is None:
                print(f"{thread_name}: ERROR - Failed to create model for {genome['id']}")
                return 0.0
            
            optimizer_class = OPTIMIZERS[genome['optimizer']]
            optimizer = optimizer_class(model.parameters(), lr=genome['learning_rate'])
            criterion = nn.CrossEntropyLoss()

            best_acc = 0.0
            best_epoch = -1
            patience_left = self.config['epoch_patience']
            last_improvement_acc = 0.0

            max_epochs = self.config['num_epochs']
            print(f"{thread_name}: Training/Evaluating model {genome['id']} (max {max_epochs} epochs interleaved)")

            for epoch in range(1, max_epochs + 1):
                # Check time limit
                elapsed_time = time.time() - evaluation_start
                if elapsed_time > max_evaluation_time:
                    print(f"{thread_name}: TIMEOUT - Stopping evaluation after {elapsed_time:.1f}s (limit: {max_evaluation_time}s)")
                    break
                
                try:
                    # 1) Train epoch with error handling
                    train_loss = self._train_one_epoch(model, optimizer, criterion, genome['id'], epoch, thread_name)
                    if train_loss is None or np.isnan(train_loss) or np.isinf(train_loss):
                        print(f"{thread_name}: ERROR - Invalid training loss, stopping evaluation")
                        break
                    
                    # 2) Evaluate right after training epoch
                    acc, eval_loss = self._evaluate(model, criterion, genome['id'], epoch, thread_name)
                    if acc is None or np.isnan(acc) or np.isinf(acc):
                        print(f"{thread_name}: ERROR - Invalid accuracy, stopping evaluation")
                        break

                    # Early stopping logic based on accuracy improvement
                    improvement = acc - last_improvement_acc
                    if improvement >= self.config['improvement_threshold']:
                        patience_left = self.config['epoch_patience']
                        last_improvement_acc = acc
                    else:
                        patience_left -= 1

                    if acc > best_acc:
                        best_acc = acc
                        best_epoch = epoch

                    print(f"{thread_name}: -> Acc={acc:.2f}% (best={best_acc:.2f}% at epoch {best_epoch}) patience_left={patience_left}")

                    if patience_left <= 0:
                        print(f"{thread_name}: Early stopping triggered (no significant improvement)")
                        break
                        
                except Exception as epoch_error:
                    print(f"{thread_name}: ERROR in epoch {epoch}: {epoch_error}")
                    print(f"{thread_name}: Continuing to next epoch...")
                    continue

            total_time = time.time() - evaluation_start
            print(f"{thread_name}: Final fitness for {genome['id']}: {best_acc:.2f}% (best epoch {best_epoch}) in {total_time:.1f}s")
            
            # Clean up model to free memory
            del model, optimizer
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            
            return best_acc
            
        except Exception as e:
            total_time = time.time() - evaluation_start
            print(f"{thread_name}: CRITICAL ERROR evaluating genome {genome['id']} after {total_time:.1f}s: {e}")
            print(f"{thread_name}: Exception type: {type(e).__name__}")
            logger.warning(f"Error evaluating genome {genome['id']}: {e}")
            
            # Clean up in case of error
            try:
                torch.cuda.empty_cache() if torch.cuda.is_available() else None
            except:
                pass
            
            return 0.0

    def _validate_genome(self, genome: dict) -> bool:
        """Validates genome structure to prevent errors."""
        required_keys = ['num_conv_layers', 'num_fc_layers', 'filters', 'kernel_sizes', 
                        'fc_nodes', 'activations', 'dropout_rate', 'learning_rate', 'optimizer']
        
        for key in required_keys:
            if key not in genome:
                return False
        
        # Basic sanity checks
        if genome['num_conv_layers'] <= 0 or genome['num_fc_layers'] <= 0:
            return False
        if len(genome['filters']) != genome['num_conv_layers']:
            return False
        if len(genome['fc_nodes']) != genome['num_fc_layers']:
            return False
        if genome['dropout_rate'] < 0 or genome['dropout_rate'] > 1:
            return False
        if genome['learning_rate'] <= 0:
            return False
            
        return True

    def _worker_evaluate_genome(self, genome: dict, index: int, total: int, results: list):
        """Worker function that evaluates a single genome in a thread with semaphore control and robust error handling."""
        thread_name = f"Thread-{index+1}"
        fitness = 0.0
        
        # Acquire semaphore to limit concurrent training
        semaphore_acquired = False
        try:
            # Timeout for semaphore acquisition (avoid deadlock)
            semaphore_acquired = self.training_semaphore.acquire(timeout=30)
            if not semaphore_acquired:
                print(f"{thread_name}: ERROR - Could not acquire semaphore within 30s")
                with self.fitness_lock:
                    genome['fitness'] = 0.0
                    results[index] = 0.0
                return
            
            print(f"{thread_name}: iniciando evaluación (máx {self.max_concurrent_training} concurrentes)...")
            print(f"{thread_name}: Evaluating individual {index+1}/{total} (ID: {genome['id']})")
            print(f"{thread_name}: Architecture: {genome['num_conv_layers']} conv + {genome['num_fc_layers']} fc, opt={genome['optimizer']}, lr={genome['learning_rate']}")
            
            # Set a timeout for the entire evaluation process
            evaluation_start = time.time()
            max_evaluation_time = 240  # 4 minutes max per model
            
            # Evaluate fitness with timeout protection
            try:
                fitness = self.evaluate_fitness(genome, thread_name)
                evaluation_time = time.time() - evaluation_start
                
                if evaluation_time > max_evaluation_time:
                    print(f"{thread_name}: WARNING - Evaluation took {evaluation_time:.1f}s (exceeded {max_evaluation_time}s)")
                
            except Exception as eval_error:
                print(f"{thread_name}: ERROR during fitness evaluation: {eval_error}")
                print(f"{thread_name}: Setting fitness to 0.0 due to evaluation error")
                fitness = 0.0
            
            # Thread-safe update of results
            with self.fitness_lock:
                genome['fitness'] = fitness
                results[index] = fitness
                print(f"{thread_name}: Fitness obtained: {fitness:.2f}%")
                
        except Exception as e:
            print(f"{thread_name}: CRITICAL ERROR durante evaluación: {e}")
            print(f"{thread_name}: Exception type: {type(e).__name__}")
            with self.fitness_lock:
                genome['fitness'] = 0.0
                results[index] = 0.0
        finally:
            if semaphore_acquired:
                print(f"{thread_name}: terminado, liberando slot de entrenamiento.")
                self.training_semaphore.release()
            else:
                print(f"{thread_name}: terminado SIN liberar semáforo (no se pudo adquirir).")

    def evaluate_population(self):
        """Thread-safe population evaluation with concurrent model training and timeout protection."""
        print(f"\nEvaluating population with threading (Generation {self.generation})...")
        print(f"Processing {len(self.population)} individuals with max {self.max_concurrent_training} concurrent training...")
        
        # Initialize results list
        fitness_scores = [0.0] * len(self.population)
        best_fitness_so_far = 0.0
        
        # Create and start threads
        threads = []
        for i, genome in enumerate(self.population):
            thread = threading.Thread(
                target=self._worker_evaluate_genome,
                args=(genome, i, len(self.population), fitness_scores)
            )
            thread.daemon = True  # Make threads daemon so they don't block program exit
            threads.append(thread)
        
        # Start all threads
        for thread in threads:
            thread.start()
        
        # Wait for all threads to complete with timeout
        timeout_per_thread = 300  # 5 minutes per thread max
        max_total_timeout = timeout_per_thread * 2  # Total timeout for all threads
        start_time = time.time()
        
        completed_threads = 0
        failed_threads = []
        
        for i, thread in enumerate(threads):
            remaining_time = max_total_timeout - (time.time() - start_time)
            if remaining_time <= 0:
                print(f"WARNING: Global timeout reached, remaining threads will be abandoned")
                break
                
            thread_timeout = min(timeout_per_thread, remaining_time)
            thread.join(timeout=thread_timeout)
            
            if thread.is_alive():
                print(f"WARNING: Thread-{i+1} timed out after {thread_timeout:.1f}s - marking as failed")
                failed_threads.append(i)
                # Set failed thread result to 0.0
                with self.fitness_lock:
                    if i < len(self.population):
                        self.population[i]['fitness'] = 0.0
                        fitness_scores[i] = 0.0
            else:
                completed_threads += 1
                print(f"Thread-{i+1} completed successfully ({completed_threads}/{len(threads)})")
        
        # Check for any remaining alive threads
        alive_threads = [i for i, thread in enumerate(threads) if thread.is_alive()]
        if alive_threads:
            print(f"WARNING: {len(alive_threads)} threads still alive after timeout: {alive_threads}")
            print("These threads will be abandoned and their fitness set to 0.0")
            for i in alive_threads:
                with self.fitness_lock:
                    if i < len(self.population):
                        self.population[i]['fitness'] = 0.0
                        fitness_scores[i] = 0.0
        
        print(f"Threading summary: {completed_threads}/{len(threads)} completed, {len(failed_threads)} failed/timed out")
        
        # Calculate generation statistics
        if fitness_scores:
            avg_fitness = np.mean(fitness_scores)
            max_fitness = np.max(fitness_scores)
            min_fitness = np.min(fitness_scores)
            std_fitness = np.std(fitness_scores)
        else:
            avg_fitness = max_fitness = min_fitness = std_fitness = 0.0

        stats = {
            'generation': self.generation,
            'avg_fitness': avg_fitness,
            'max_fitness': max_fitness,
            'min_fitness': min_fitness,
            'std_fitness': std_fitness
        }
        self.generation_stats.append(stats)
        self.fitness_history.append(max_fitness)

        best_genome = max(self.population, key=lambda x: x['fitness'])
        if self.best_individual is None or best_genome['fitness'] > self.best_individual['fitness']:
            self.best_individual = copy.deepcopy(best_genome)
            print(f"\nNew global best individual found!")

        print(f"\nGENERATION {self.generation} STATISTICS (Threading completed):")
        print(f"   Maximum fitness: {max_fitness:.2f}%")
        print(f"   Average fitness: {avg_fitness:.2f}%")
        print(f"   Minimum fitness: {min_fitness:.2f}%")
        print(f"   Standard deviation: {std_fitness:.2f}%")
        print(f"   Best individual: {best_genome['id']} with {best_genome['fitness']:.2f}%")
        print(f"   Global best individual: {self.best_individual['id']} with {self.best_individual['fitness']:.2f}%")
        print(f"   Concurrent training slots used: {self.max_concurrent_training}")

    def selection_and_reproduction(self):
        print(f"\nStarting selection and reproduction...")
        # Sort by fitness
        self.population.sort(key=lambda x: x['fitness'], reverse=True)
        elite_size = max(1, int(self.config['population_size'] * self.config['elite_percentage']))
        elite = self.population[:elite_size]
        print(f"Selecting {elite_size} elite individuals:")
        for i, individual in enumerate(elite):
            print(f"   Elite {i+1}: {individual['id']} (fitness: {individual['fitness']:.2f}%)")
        new_population = copy.deepcopy(elite)
        offspring_needed = self.config['population_size'] - len(new_population)
        print(f"Creating {offspring_needed} new individuals through crossover and mutation...")
        offspring_created = 0
        while len(new_population) < self.config['population_size']:
            parent1 = self.tournament_selection()
            parent2 = self.tournament_selection()
            child1, child2 = crossover_genomes(parent1, parent2, self.config)
            child1 = mutate_genome(child1, self.config)
            if len(new_population) < self.config['population_size']:
                new_population.append(child1)
            child2 = mutate_genome(child2, self.config)
            if len(new_population) < self.config['population_size']:
                new_population.append(child2)
            offspring_created += 2
            if offspring_created % 4 == 0:
                print(f"   Created {min(offspring_created, offspring_needed)} of {offspring_needed} new individuals...")
        self.population = new_population[:self.config['population_size']]
        print(f"New generation created with {len(self.population)} individuals")
        print(f"   Elite preserved: {elite_size}")
        print(f"   New individuals: {len(self.population) - elite_size}")

    def tournament_selection(self, tournament_size: int = 3) -> dict:
        tournament = random.sample(self.population, min(tournament_size, len(self.population)))
        return max(tournament, key=lambda x: x['fitness'])

    def _update_adaptive_mutation(self):
        # Diversity measured via std of fitness in last generation
        if not self.generation_stats:
            self.config['current_mutation_rate'] = self.config['base_mutation_rate']
            return
        last_std = self.generation_stats[-1]['std_fitness']
        # Heuristic: more diversity -> lower mutation, low diversity -> higher
        # Normalize std roughly assuming fitness in [0,100]
        diversity_factor = min(1.0, last_std / 10.0)  # std 10% -> factor 1
        # Invert: low diversity (small std) should raise mutation
        inverted = 1 - diversity_factor
        new_rate = self.config['base_mutation_rate'] + (inverted - 0.5) * 0.4  # adjust +/-0.2 range
        new_rate = max(self.config['mutation_rate_min'], min(self.config['mutation_rate_max'], new_rate))
        self.config['current_mutation_rate'] = round(new_rate, 4)
        print(f"Adaptive mutation rate updated to {self.config['current_mutation_rate']} (std_fitness={last_std:.2f})")

    def check_convergence(self) -> bool:
        if self.best_individual and self.best_individual['fitness'] >= self.config['fitness_threshold']:
            print(f"Target fitness reached! ({self.best_individual['fitness']:.2f}% >= {self.config['fitness_threshold']}%)")
            return True
        if self.generation >= self.config['max_generations']:
            print(f"Maximum generations reached ({self.generation}/{self.config['max_generations']})")
            return True
        if len(self.fitness_history) >= 3:
            recent = self.fitness_history[-3:]
            if max(recent) - min(recent) < 0.5:
                print("Stagnation detected in last 3 generations")
                return True
        return False

    def evolve(self) -> dict:
        print("STARTING HYBRID NEUROEVOLUTION PROCESS (adaptive mutation + threading)")
        print("="*60)
        print(f"Configuration:")
        print(f"   Population: {self.config['population_size']} individuals")
        print(f"   Maximum generations: {self.config['max_generations']}")
        print(f"   Target fitness: {self.config['fitness_threshold']}%")
        print(f"   Max concurrent training: {self.max_concurrent_training} models")
        print(f"   Device: {device}")
        print("="*60)
        self.initialize_population()
        while not self.check_convergence():
            print(f"\n{'='*80}")
            print(f"GENERATION {self.generation}")
            print(f"{'='*80}")
            self.evaluate_population()
            if self.check_convergence():
                break
            self._update_adaptive_mutation()
            self.selection_and_reproduction()
            self.generation += 1
            print(f"\nPreparing for next generation...")
        print(f"\n{'='*80}")
        print(f"EVOLUTION COMPLETED!")
        print(f"{'='*80}")
        print(f"Best individual found:")
        print(f"   ID: {self.best_individual['id']}")
        print(f"   Fitness: {self.best_individual['fitness']:.2f}%")
        print(f"   Origin generation: {self.generation}")
        print(f"   Total generations processed: {self.generation + 1}")
        print(f"   Threading efficiency: {self.max_concurrent_training} concurrent training slots")
        return self.best_individual

print("HybridNeuroevolution class updated with adaptive mutation, interleaved early stopping, and threading support")

## 7. Evolution Process Execution

In [ ]:
# ==========================================
# CONFIGURACIÓN DE DATASET - MODIFICAR AQUÍ
# ==========================================

# Para cambiar el dataset, modifica la línea correspondiente y ejecuta esta celda:

# Opción 1: Usar MNIST (28x28, grayscale, 10 classes)
# CONFIG['dataset'] = 'MNIST'

# Opción 2: Usar CIFAR-10 (32x32, RGB, 10 classes) - RECOMENDADO para mayor challenge
# CONFIG['dataset'] = 'CIFAR10'

# Opción 3: Usar dataset personalizado
# CONFIG['dataset'] = 'CUSTOM'
# CONFIG['dataset_path'] = r'E:\Neuroevolution\data\phd_data'  # Ajustar ruta según tu dataset

# ==========================================
# OTRAS CONFIGURACIONES OPCIONALES
# ==========================================


# Reconfigurar el dataset con los nuevos parámetros
CONFIG = configure_dataset(CONFIG, CONFIG['dataset'])

print("Current configuration:")
print(f"   Dataset: {CONFIG['dataset']}")
print(f"   Image size: {CONFIG['px_h']}x{CONFIG['px_w']}x{CONFIG['num_channels']}")
print(f"   Number of classes: {CONFIG['num_classes']}")
print(f"   Population: {CONFIG['population_size']} individuals")
print(f"   Maximum generations: {CONFIG['max_generations']}")
print(f"   Target fitness: {CONFIG['fitness_threshold']}%")
print(f"   Device: {device}")

# Recargar el dataset con la nueva configuración
print(f"\nReloading dataset with new configuration...")
train_loader, test_loader = load_dataset(CONFIG)

# Initialize neuroevolution system
start_time = datetime.now()
print(f"\nStarting neuroevolution at {start_time.strftime('%H:%M:%S')}")

# Create system instance
neuroevolution = HybridNeuroevolution(CONFIG, train_loader, test_loader)

# Execute evolution process
best_genome = neuroevolution.evolve()

end_time = datetime.now()
execution_time = end_time - start_time

print(f"\nProcess completed at {end_time.strftime('%H:%M:%S')}")
print(f"Total execution time: {execution_time}")
print(f"Total generations: {neuroevolution.generation}")
print(f"Best fitness achieved: {best_genome['fitness']:.2f}%")

## 8. Results Visualization and Analysis

In [ ]:
# Configure matplotlib style
plt.style.use('default')
sns.set_palette("husl")

# Function to visualize fitness evolution
def plot_fitness_evolution(neuroevolution):
    """Plots fitness evolution across generations."""
    if not neuroevolution.generation_stats:
        print("WARNING: No statistics data to plot")
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Extract data and filter 0.00 fitness
    generations = []
    avg_fitness = []
    max_fitness = []
    min_fitness = []
    std_fitness = []
    
    for stat in neuroevolution.generation_stats:
        # Only include if valid fitness (> 0.00)
        if stat['max_fitness'] > 0.00:
            generations.append(stat['generation'])
            avg_fitness.append(stat['avg_fitness'])
            max_fitness.append(stat['max_fitness'])
            min_fitness.append(stat['min_fitness'])
            std_fitness.append(stat['std_fitness'])
    
    if not generations:
        print("WARNING: No valid fitness data to plot (all are 0.00)")
        return
    
    # Graph 1: Fitness evolution
    ax1.plot(generations, max_fitness, 'g-', linewidth=2, marker='o', label='Maximum Fitness')
    ax1.plot(generations, avg_fitness, 'b-', linewidth=2, marker='s', label='Average Fitness')
    ax1.plot(generations, min_fitness, 'r-', linewidth=2, marker='^', label='Minimum Fitness')
    ax1.fill_between(generations, 
                     [max(0, avg - std) for avg, std in zip(avg_fitness, std_fitness)],
                     [avg + std for avg, std in zip(avg_fitness, std_fitness)],
                     alpha=0.2, color='blue')
    
    ax1.set_xlabel('Generation')
    ax1.set_ylabel('Fitness (%)')
    ax1.set_title('Fitness Evolution by Generation (Excluding 0.00%)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add target fitness line
    ax1.axhline(y=CONFIG['fitness_threshold'], color='orange', linestyle='--', 
                label=f"Target ({CONFIG['fitness_threshold']}%)")
    ax1.legend()
    
    # Set Y axis limits for better visualization
    y_min = max(0, min(min_fitness) - 5)
    y_max = min(100, max(max_fitness) + 5)
    ax1.set_ylim(y_min, y_max)
    
    # Graph 2: Diversity (standard deviation)
    ax2.plot(generations, std_fitness, 'purple', linewidth=2, marker='D')
    ax2.set_xlabel('Generation')
    ax2.set_ylabel('Fitness Standard Deviation')
    ax2.set_title('Population Diversity (Excluding 0.00%)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Show additional information
    print(f"Plotted data:")
    print(f"   Generations with valid fitness: {len(generations)}")
    print(f"   Best fitness achieved: {max(max_fitness):.2f}%")
    print(f"   Final average fitness: {avg_fitness[-1]:.2f}%")
    if len(generations) < len(neuroevolution.generation_stats):
        excluded = len(neuroevolution.generation_stats) - len(generations)
        print(f"   WARNING: Excluded generations (0.00 fitness): {excluded}")

# Function to show detailed statistics
def show_evolution_statistics(neuroevolution):
    """Shows detailed evolution statistics."""
    print("DETAILED EVOLUTION STATISTICS")
    print("="*60)
    
    if not neuroevolution.generation_stats:
        print("WARNING: No statistics available")
        return
    
    # Filter statistics with valid fitness
    valid_stats = [stat for stat in neuroevolution.generation_stats if stat['max_fitness'] > 0.00]
    
    if not valid_stats:
        print("WARNING: No valid statistics (all fitness are 0.00)")
        return
    
    final_stats = valid_stats[-1]
    
    print(f"Completed generations: {neuroevolution.generation}")
    print(f"Generations with valid fitness: {len(valid_stats)}")
    if len(valid_stats) < len(neuroevolution.generation_stats):
        excluded = len(neuroevolution.generation_stats) - len(valid_stats)
        print(f"WARNING: Generations with 0.00 fitness (excluded): {excluded}")
    
    print(f"\nFINAL STATISTICS (excluding 0.00 fitness):")
    print(f"   Final best fitness: {final_stats['max_fitness']:.2f}%")
    print(f"   Final average fitness: {final_stats['avg_fitness']:.2f}%")
    print(f"   Final minimum fitness: {final_stats['min_fitness']:.2f}%")
    print(f"   Final standard deviation: {final_stats['std_fitness']:.2f}%")
    
    # Progress across generations
    if len(valid_stats) > 1:
        initial_max = valid_stats[0]['max_fitness']
        final_max = valid_stats[-1]['max_fitness']
        improvement = final_max - initial_max
        
        print(f"\nPROGRESS:")
        print(f"   Initial fitness: {initial_max:.2f}%")
        print(f"   Final fitness: {final_max:.2f}%")
        print(f"   Total improvement: {improvement:.2f}%")
        if initial_max > 0:
            print(f"   Relative improvement: {(improvement/initial_max)*100:.1f}%")
    
    # Convergence analysis
    print(f"\nCONVERGENCE CRITERIA:")
    if neuroevolution.best_individual and neuroevolution.best_individual['fitness'] >= CONFIG['fitness_threshold']:
        print(f"   OK: Target fitness reached ({CONFIG['fitness_threshold']}%)")
    else:
        print(f"   ERROR: Target fitness NOT reached ({CONFIG['fitness_threshold']}%)")
    
    if neuroevolution.generation >= CONFIG['max_generations']:
        print(f"   TIME: Maximum generations reached ({CONFIG['max_generations']})")
    
    # Additional performance statistics
    all_max_fitness = [stat['max_fitness'] for stat in valid_stats]
    all_avg_fitness = [stat['avg_fitness'] for stat in valid_stats]
    
    print(f"\nGENERAL STATISTICS:")
    print(f"   Best fitness of entire evolution: {max(all_max_fitness):.2f}%")
    print(f"   Average fitness of entire evolution: {np.mean(all_avg_fitness):.2f}%")
    print(f"   Average improvement per generation: {(max(all_max_fitness) - min(all_max_fitness))/len(valid_stats):.2f}%")
    
    if neuroevolution.best_individual:
        print(f"\nBest individual ID: {neuroevolution.best_individual['id']}")
        print(f"Best individual fitness: {neuroevolution.best_individual['fitness']:.2f}%")

# Additional function for failure analysis
def analyze_failed_evaluations(neuroevolution):
    """Analyzes evaluations that resulted in 0.00 fitness."""
    print("\nFAILED EVALUATIONS ANALYSIS")
    print("="*50)
    
    total_generations = len(neuroevolution.generation_stats)
    failed_generations = len([stat for stat in neuroevolution.generation_stats if stat['max_fitness'] == 0.00])
    
    if failed_generations == 0:
        print("OK: No failed evaluations (0.00 fitness)")
        return
    
    success_rate = ((total_generations - failed_generations) / total_generations) * 100
    
    print(f"Failure summary:")
    print(f"   Total generations: {total_generations}")
    print(f"   Failed generations: {failed_generations}")
    print(f"   Success rate: {success_rate:.1f}%")
    
    if failed_generations > 0:
        failed_gens = [stat['generation'] for stat in neuroevolution.generation_stats if stat['max_fitness'] == 0.00]
        print(f"   Generations with failures: {failed_gens}")
        
        print(f"\nPossible causes of 0.00 fitness:")
        print(f"   • Errors in model architecture")
        print(f"   • Memory problems (GPU/RAM)")
        print(f"   • Invalid hyperparameter configurations")
        print(f"   • Errors during training")

# Execute visualizations
plot_fitness_evolution(neuroevolution)
show_evolution_statistics(neuroevolution)
analyze_failed_evaluations(neuroevolution)

## 9. BEST ARCHITECTURE FOUND

In [ ]:
def display_best_architecture(best_genome, config):
    """
    Shows the best architecture found in detailed and visual format.
    """
    print("="*60)
    print("        BEST EVOLVED ARCHITECTURE")
    print("="*60)
    
    # General information
    print(f"\nGENERAL INFORMATION:")
    print(f"   Genome ID: {best_genome['id']}")
    print(f"   Fitness Achieved: {best_genome['fitness']:.2f}%")
    print(f"   Generation: {neuroevolution.generation}")
    
    # Architecture details
    print(f"\nNETWORK ARCHITECTURE:")
    print(f"   Convolutional Layers: {best_genome['num_conv_layers']}")
    print(f"   Fully Connected Layers: {best_genome['num_fc_layers']}")
    
    print(f"\nCONVOLUTIONAL LAYER DETAILS:")
    for i in range(best_genome['num_conv_layers']):
        filters = best_genome['filters'][i]
        kernel = best_genome['kernel_sizes'][i]
        activation = best_genome['activations'][i % len(best_genome['activations'])]
        print(f"   Conv{i+1}: {filters} filters, kernel {kernel}x{kernel}, activation {activation}")
    
    print(f"\nFULLY CONNECTED LAYER DETAILS:")
    for i, nodes in enumerate(best_genome['fc_nodes']):
        print(f"   FC{i+1}: {nodes} neurons")
    print(f"   Output: {config['num_classes']} neurons (classes)")
    
    print(f"\nHYPERPARAMETERS:")
    print(f"   Optimizer: {best_genome['optimizer'].upper()}")
    print(f"   Learning Rate: {best_genome['learning_rate']:.4f}")
    print(f"   Dropout Rate: {best_genome['dropout_rate']:.3f}")
    print(f"   Activation Functions: {', '.join(best_genome['activations'])}")
    
    # Create and show final model
    print(f"\nCREATING FINAL MODEL...")
    try:
        final_model = EvolvableCNN(best_genome, config)
        total_params = sum(p.numel() for p in final_model.parameters())
        trainable_params = sum(p.numel() for p in final_model.parameters() if p.requires_grad)
        
        print(f"   Model created successfully")
        print(f"   Total parameters: {total_params:,}")
        print(f"   Trainable parameters: {trainable_params:,}")
        
        # Architecture summary
        print(f"\nCOMPACT SUMMARY:")
        print(f"   {final_model.get_architecture_summary()}")
        
    except Exception as e:
        print(f"   ERROR creating model: {e}")
    
    # Visualization in table format
    print(f"\nSUMMARY TABLE:")
    print(f"{'='*80}")
    print(f"{'Parameter':<25} {'Value':<30} {'Description':<25}")
    print(f"{'='*80}")
    print(f"{'ID':<25} {best_genome['id']:<30} {'Unique identifier':<25}")
    print(f"{'Fitness':<25} {best_genome['fitness']:.2f}%{'':<25} {'Accuracy achieved':<25}")
    print(f"{'Conv Layers':<25} {best_genome['num_conv_layers']:<30} {'Convolutional layers':<25}")
    print(f"{'FC Layers':<25} {best_genome['num_fc_layers']:<30} {'FC layers':<25}")
    print(f"{'Optimizer':<25} {best_genome['optimizer']:<30} {'Optimization algorithm':<25}")
    print(f"{'Learning Rate':<25} {best_genome['learning_rate']:<30} {'Learning rate':<25}")
    print(f"{'Dropout':<25} {best_genome['dropout_rate']:<30} {'Dropout rate':<25}")
    print(f"{'='*80}")
    
    # Comparison with initial configuration
    print(f"\nCOMPARISON WITH OBJECTIVES:")
    if best_genome['fitness'] >= config['fitness_threshold']:
        print(f"   TARGET: OK Fitness objective REACHED ({best_genome['fitness']:.2f}% >= {config['fitness_threshold']}%)")
    else:
        print(f"   TARGET: ERROR Fitness objective NOT reached ({best_genome['fitness']:.2f}% < {config['fitness_threshold']}%)")
    
    print(f"   TIME: Generations used: {neuroevolution.generation}/{config['max_generations']}")
    
    # Save information to JSON
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"best_architecture_{timestamp}.json"
    
    results_data = {
        'timestamp': timestamp,
        'execution_time': str(execution_time),
        'config_used': config,
        'best_genome': best_genome,
        'final_generation': neuroevolution.generation,
        'evolution_stats': neuroevolution.generation_stats
    }
    
    try:
        with open(results_file, 'w') as f:
            json.dump(results_data, f, indent=2, default=str)
        print(f"\nResults saved to: {results_file}")
    except Exception as e:
        print(f"\nWARNING: Error saving results: {e}")
    
    print(f"\nHYBRID NEUROEVOLUTION COMPLETED SUCCESSFULLY!")
    print(f"{'='*60}")

# Show the best architecture found
display_best_architecture(best_genome, CONFIG)

# Demostración de Threading en Hybrid Neuroevolution

Este notebook ha sido modificado para implementar **entrenamiento concurrente** de modelos usando threading, similar al patrón mostrado en `threading_demo_notebook.ipynb`.

## Principales modificaciones implementadas:

### 1. **Threading Support en HybridNeuroevolution**
- Agregado soporte para entrenamiento concurrente de múltiples modelos
- Máximo de 2 modelos entrenando simultáneamente (configurable)
- Thread-safety usando semáforos y locks

### 2. **Nuevos componentes de threading**
- `training_semaphore`: Limita el número de entrenamientos concurrentes
- `fitness_lock`: Protege el acceso a variables compartidas
- `_worker_evaluate_genome()`: Función worker para evaluación en hilos separados
- Threading-aware logging con identificadores de thread

### 3. **Método evaluate_population() rediseñado**
- Reemplaza evaluación secuencial con evaluación concurrente
- Crea threads para cada genoma en la población
- Sincronización segura de resultados de fitness
- Manejo de errores por thread individual

### 4. **Beneficios del threading**
- **Paralelización**: Múltiples modelos entrenan simultáneamente
- **Eficiencia**: Mejor uso de recursos computacionales
- **Escalabilidad**: Fácil ajuste del número de threads concurrentes
- **Robustez**: Manejo seguro de estado compartido

### 5. **Configuración del threading**
```python
self.max_concurrent_training = 2  # Máximo 2 modelos concurrentes
self.training_semaphore = threading.Semaphore(2)
self.fitness_lock = threading.Lock()
```

El patrón implementado es idéntico al del notebook de demostración, adaptado para el contexto de neuroevolución híbrida.

In [ ]:
# Prueba del sistema de threading con población pequeña
print("="*60)
print("PRUEBA DE THREADING CON POBLACIÓN PEQUEÑA")
print("="*60)

# Configuración reducida para prueba rápida
TEST_CONFIG = {
    # Dataset configuration
    'dataset': 'MNIST',
    'num_classes': 10,
    'num_channels': 1,
    'px_h': 28,
    'px_w': 28,
    
    # Architecture constraints
    'min_conv_layers': 1,
    'max_conv_layers': 2,
    'min_fc_layers': 1,
    'max_fc_layers': 1,
    'min_filters': 8,
    'max_filters': 16,
    'min_fc_nodes': 32,
    'max_fc_nodes': 64,
    
    # Evolution parameters
    'population_size': 4,  # Población pequeña para prueba
    'max_generations': 2,  # Solo 2 generaciones
    'fitness_threshold': 85.0,
    'elite_percentage': 0.5,
    'crossover_rate': 0.8,
    
    # Mutation parameters (adaptive)
    'base_mutation_rate': 0.2,
    'current_mutation_rate': 0.2,
    'mutation_rate_min': 0.1,
    'mutation_rate_max': 0.6,
    
    # Training parameters (reducidos para prueba rápida)
    'num_epochs': 2,  # Solo 2 épocas por modelo
    'early_stopping_patience': 50,
    'epoch_patience': 5,
    'improvement_threshold': 0.5
}

# Crear datasets de prueba (tamaño reducido)
print("Creando datasets de prueba...")
train_dataset = datasets.MNIST('./data', train=True, download=True,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                              ]))

test_dataset = datasets.MNIST('./data', train=False, download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.1307,), (0.3081,))
                             ]))

# Usar solo un subconjunto para prueba rápida
train_subset = torch.utils.data.Subset(train_dataset, range(1000))  # Solo 1000 muestras
test_subset = torch.utils.data.Subset(test_dataset, range(200))     # Solo 200 muestras

test_train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

print(f"Dataset de prueba creado: {len(train_subset)} train, {len(test_subset)} test")

# Crear instancia del algoritmo evolutivo con threading
print("Iniciando algoritmo evolutivo con threading...")
test_neuroevolution = HybridNeuroevolution(TEST_CONFIG, test_train_loader, test_test_loader)

# Ejecutar evolución de prueba
print("Ejecutando evolución de prueba...")
import time
start_time = time.time()

best_test_genome = test_neuroevolution.evolve()

end_time = time.time()
execution_time_test = end_time - start_time

print(f"\n{'='*80}")
print(f"PRUEBA DE THREADING COMPLETADA")
print(f"{'='*80}")
print(f"Tiempo de ejecución: {execution_time_test:.2f} segundos")
print(f"Mejor individuo encontrado:")
print(f"   ID: {best_test_genome['id']}")
print(f"   Fitness: {best_test_genome['fitness']:.2f}%")
print(f"   Generaciones procesadas: {test_neuroevolution.generation + 1}")
print(f"   Threading slots concurrentes: {test_neuroevolution.max_concurrent_training}")
print(f"\nLa prueba demuestra que el threading funciona correctamente!")
print(f"Se entrenaron máximo {test_neuroevolution.max_concurrent_training} modelos concurrentemente.")
print("="*80)

## Configuración de Threading y Timeouts

Las siguientes mejoras han sido implementadas para resolver el problema de threads que se quedan colgados:

### 🕐 **Timeouts implementados:**
- **Thread timeout**: 5 minutos por thread individual
- **Global timeout**: Tiempo total máximo para toda la población
- **Evaluation timeout**: 3 minutos máximo por evaluación de fitness
- **Semaphore timeout**: 30 segundos para adquirir slot de entrenamiento

### 🛡️ **Manejo robusto de errores:**
- Validación de genomas antes de crear modelos
- Detección de valores NaN/Inf en loss y accuracy
- Liberación automática de memoria GPU
- Threads daemon para evitar bloqueo del programa
- Manejo de excepciones por batch individual

### ⚙️ **Configuración flexible:**
Puedes ajustar estos parámetros en la clase `HybridNeuroevolution`:

```python
# En __init__:
self.max_concurrent_training = config.get('max_concurrent_training', 2)

# En evaluate_population:
timeout_per_thread = 300  # 5 minutos por thread
max_total_timeout = timeout_per_thread * 2  # Timeout total

# En evaluate_fitness:
max_evaluation_time = 180  # 3 minutos por evaluación
```

### 📊 **Monitoreo mejorado:**
- Logs detallados por thread con timestamps
- Conteo de threads completados vs fallidos
- Identificación de threads que exceden timeout
- Limpieza automática de threads abandonados

In [ ]:
# Prueba del sistema de threading CON TIMEOUTS - Más robusta
print("="*70)
print("PRUEBA DE THREADING CON TIMEOUTS Y MANEJO ROBUSTO DE ERRORES")
print("="*70)

# Configuración de prueba con threading y timeouts
ROBUST_TEST_CONFIG = {
    # Dataset configuration
    'dataset': 'MNIST',
    'num_classes': 10,
    'num_channels': 1,
    'px_h': 28,
    'px_w': 28,
    
    # Architecture constraints (más permisivas para generar algunos errores)
    'min_conv_layers': 1,
    'max_conv_layers': 3,
    'min_fc_layers': 1,
    'max_fc_layers': 2,
    'min_filters': 4,    # Muy pocos filtros para causar posibles errores
    'max_filters': 32,
    'min_fc_nodes': 16,  # Pocos nodos para causar posibles errores
    'max_fc_nodes': 128,
    
    # Evolution parameters
    'population_size': 6,  # 6 individuos para probar timeouts
    'max_generations': 1,  # Solo 1 generación para prueba
    'fitness_threshold': 90.0,
    'elite_percentage': 0.5,
    'crossover_rate': 0.8,
    
    # Threading configuration - AQUÍ PUEDES AJUSTAR EL NÚMERO DE THREADS
    'max_concurrent_training': 2,  # ← CAMBIA ESTE VALOR PARA MÁS/MENOS THREADS
    
    # Mutation parameters
    'base_mutation_rate': 0.3,  # Alta mutación para generar arquitecturas problemáticas
    'current_mutation_rate': 0.3,
    'mutation_rate_min': 0.1,
    'mutation_rate_max': 0.7,
    
    # Training parameters (reducidos para prueba rápida)
    'num_epochs': 2,
    'early_stopping_patience': 30,  # Pocos batches para acelerar
    'epoch_patience': 3,
    'improvement_threshold': 1.0
}

print(f"Configuración de threading:")
print(f"   Threads concurrentes: {ROBUST_TEST_CONFIG['max_concurrent_training']}")
print(f"   Población: {ROBUST_TEST_CONFIG['population_size']} individuos")
print(f"   Timeout por thread: 5 minutos")
print(f"   Timeout total: {5 * 2} minutos")

# Crear datasets pequeños para prueba rápida
print("\nCreando datasets pequeños para prueba...")
if 'train_dataset' not in locals():
    train_dataset = datasets.MNIST('./data', train=True, download=True,
                                  transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,))
                                  ]))
    test_dataset = datasets.MNIST('./data', train=False, download=True,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.1307,), (0.3081,))
                                 ]))

# Usar subconjuntos aún más pequeños
robust_train_subset = torch.utils.data.Subset(train_dataset, range(500))  # Solo 500 muestras
robust_test_subset = torch.utils.data.Subset(test_dataset, range(100))    # Solo 100 muestras

robust_train_loader = DataLoader(robust_train_subset, batch_size=32, shuffle=True)
robust_test_loader = DataLoader(robust_test_subset, batch_size=32, shuffle=False)

print(f"Datasets creados: {len(robust_train_subset)} train, {len(robust_test_subset)} test")

# Inicializar el sistema con timeouts
print(f"\n{'='*70}")
print("INICIANDO NEUROEVOLUCIÓN CON THREADING ROBUSTO")
print(f"{'='*70}")

robust_neuroevolution = HybridNeuroevolution(ROBUST_TEST_CONFIG, robust_train_loader, robust_test_loader)

# Mostrar configuración de threading actual
print(f"\nConfiguración de threading aplicada:")
print(f"   max_concurrent_training: {robust_neuroevolution.max_concurrent_training}")
print(f"   Semaphore slots: {robust_neuroevolution.training_semaphore._value}")

# Ejecutar evolución con timeouts
print(f"\nEjecutando evolución robusta...")
robust_start_time = time.time()

try:
    robust_best_genome = robust_neuroevolution.evolve()
    robust_success = True
except Exception as e:
    print(f"ERROR durante evolución: {e}")
    robust_success = False
    robust_best_genome = None

robust_end_time = time.time()
robust_execution_time = robust_end_time - robust_start_time

print(f"\n{'='*70}")
print(f"PRUEBA ROBUSTA DE THREADING COMPLETADA")
print(f"{'='*70}")
print(f"Tiempo de ejecución: {robust_execution_time:.2f} segundos")
print(f"Éxito: {'SÍ' if robust_success else 'NO'}")

if robust_success and robust_best_genome:
    print(f"Mejor individuo encontrado:")
    print(f"   ID: {robust_best_genome['id']}")
    print(f"   Fitness: {robust_best_genome['fitness']:.2f}%")
    print(f"   Generaciones procesadas: {robust_neuroevolution.generation + 1}")
else:
    print(f"No se completó exitosamente, pero el sistema no se colgó!")

print(f"\n🎉 RESULTADO: El sistema de threading con timeouts funciona correctamente!")
print(f"📊 Threads concurrentes configurados: {robust_neuroevolution.max_concurrent_training}")
print(f"⏱️  Sistema robusto contra threads colgados y arquitecturas problemáticas")
print("="*70)